In [109]:
from datasets import load_from_disk, load_dataset, DatasetDict, Dataset, concatenate_datasets
import os
os.chdir('/workspace/FutureGPT2/src/')
from data.utils import get_tokenizer, get_token_dict
import secrets
import numpy as np
from itertools import islice

In [114]:
dataset = load_from_disk('/workspace/corpus/msmarco/msmarco_LLAMA2_64tokens_1m/val')

In [2]:
# commented subsets <100k rows
SUBSETS = [
    'ArXiv',
    #'BookCorpus2',
    #'Books3',
    'DM Mathematics',
    'Enron Emails',
    #'EuroParl',
    'FreeLaw',
    'Github',
    #'Gutenberg (PG-19)',
    'HackerNews',
    'NIH ExPorter',
    #'OpenSubtitles',
    'OpenWebText2',
    #'PhilPapers',
    'Pile-CC',
    'PubMed Abstracts',
    'PubMed Central',
    #'StackExchange',
    'UPSTO Backgrounds',
    #'Ubuntu IRC',
    'Wikipedia (en)',
    'YoutubeSubtitles',
]

In [49]:
secrets.token_hex(16)

'065747e4dd8c63bf1b878054fe0f00c4'

In [3]:
MODEL_NAME_SHORT = 'MISTRAL'
MODEL_NAME = {
    'GPT2': 'gpt2',
    'LLAMA2': 'daryl149/llama-2-7b-hf',
    'MISTRAL': 'mistralai/Mistral-7B-v0.1',
}[MODEL_NAME_SHORT]
SAVE_PATH = '/workspace/corpus/msmarco/'
MAX_LENGTH = 64

In [4]:
ds = load_dataset('ArmelR/the-pile-splitted', 'ArXiv', split='train', streaming=True)

Resolving data files:   0%|          | 0/3408 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2112 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/222 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/96 [00:00<?, ?it/s]

In [134]:
tokenizer = get_tokenizer(MODEL_NAME)

def tokenize_chunks(examples, seq_length=64, max_chunks=5):
    tokens = tokenizer(
        examples['text'],
        truncation=True,
        max_length=seq_length * max_chunks,
        padding=False,
        add_special_tokens=False,   # hopefully it's fine not to have bod tokens...
    )
    chunks = {
        'input_ids': [],
        'attention_mask': [],
        'text': [],   # decoded text for convenience
        'id': [],     # unique hex string for convenience
    }
    for input_ids, att_mask in zip(tokens['input_ids'], tokens['attention_mask']):
        while len(input_ids) >= seq_length:
            cur_ids, input_ids = input_ids[:seq_length], input_ids[seq_length:]
            cur_mask, att_mask = att_mask[:seq_length], att_mask[seq_length:]
            chunks['input_ids'].append(cur_ids)
            chunks['attention_mask'].append(cur_mask)
            chunks['text'].append(tokenizer.decode(cur_ids))
            chunks['id'].append(secrets.token_hex(16))
    return chunks

In [135]:
ds_iter = iter(ds)

In [137]:
tokenized = Dataset.from_list([])
while len(tokenized) < 10000:
    batch = Dataset.from_list([next(ds_iter) for _ in range(100)])
    tokenized = concatenate_datasets([
        tokenized, 
        batch.map(tokenize_chunks, batched=True, batch_size=100, remove_columns=['text', 'meta', 'domain'])
    ])
    print(len(tokenized))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

494


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

990


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

...

9370


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

9863


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

10347


In [140]:
a = iter(tokenized['text'])

In [171]:
next(a)

"---\nabstract: 'Spectral triples (of compact type) are constructed on arbitrary separable quasidiagonal $C^*$-algebras. On the other hand an example of a spectral triple on a non-quasidiagonal algebra is presented.'\naddress:\n- 'Department of"

In [70]:
tokenizer.eos_token_id in a['input_ids'][0]

False

In [86]:
a['input_ids'][0][0]

11940

In [72]:
tokenizer.eos_token

'</s>'

In [73]:
tokenizer.decode([1])

'<s>'

In [63]:
[0, 1, 2, 0].strip(0)

AttributeError: 'list' object has no attribute 'strip'

In [67]:
np.mean(a['attention_mask'][0])

1.0

In [42]:
len(a['input_ids'][0])

10661

In [ ]:
def chunk(examples, seq_len=64):
    chunks = []
    while len(examples['input_ids'] >= 64):
        cur_ids, remain_

In [45]:
a.keys()

dict_keys(['input_ids', 'attention_mask'])

In [37]:
print('TOKENIZE')
tokenized = ds[:100].map(tokenize_text, batched=True, batch_size=1024)
print(f'Tokenized {len(tokenized)} texts.')

TOKENIZE


AttributeError: 'dict' object has no attribute 'map'

In [9]:
len(tokenized[0]['input_ids'])

64

In [10]:
print('FILTER')
# LLAMA2 tokenizer: shorter texts are padded at the *beginning* with tokenizer.pad_token
filtered_msmarco = tokenized.filter(
    lambda x: x['input_ids'][-1] != tokenizer.pad_token_id and x['input_ids'][0] != tokenizer.pad_token_id
)
print(f'Truncated to {MAX_LENGTH} tokens and removed shorter texts.')
print(f'Filtered from {len(tokenized)} to {len(tokenized)} rows.')

FILTER


Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

Truncated to 64 tokens and removed shorter texts.
Filtered from 100000 to 100000 rows.


In [6]:
DatasetDict({'train': ds, 'test': ds}).save_to_disk('test_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/100000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [31]:
!rm -r test_dataset

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
load_from_disk('test_dataset')['train'][0]

{'text': 'this guy  cracks me up \n---------------------- Forwarded by Drew Fossum/ET&S/Enron on 05/19/2000 \n09:09 AM ---------------------------\n\n05/19/2000 08:29 AM\nLouis Soldano\nLouis Soldano\nLouis Soldano\n05/19/2000 08:29 AM\n05/19/2000 08:29 AM\nTo: Drew Fossum/ET&S/Enron@ENRON\ncc:  \n\nSubject: Re: Emma Caplan  \n\nWell, if she\'s "very bright, personable, and enthusiastic" she certainly \nwon\'t fit in with us.   \n\nI\'ll follow up with mike and shelly on the person we were thiniking of \nsplitting for the summer - that may already be a done deal, if not it may \nmake sense to do something more permanent but we will need to look at \nbudget...\n\ni could probably use some part time help with Bret Reich exiled to Phase IV \nbut i should have a licensed lawyer for that.\n\n\n\n\n   \n\t\n\t\n\tFrom:  Drew Fossum                           05/18/2000 05:50 PM\n\t\n\nTo: Michael Moran/ET&S/Enron@ENRON, Dorothy McCoppin/FGT/Enron@ENRON, Louis \nSoldano/ET&S/Enron@ENRON, Britt

In [16]:
DatasetDict({'train': ds, 'test': ds})['test'][0]

{'text': 'this guy  cracks me up \n---------------------- Forwarded by Drew Fossum/ET&S/Enron on 05/19/2000 \n09:09 AM ---------------------------\n\n05/19/2000 08:29 AM\nLouis Soldano\nLouis Soldano\nLouis Soldano\n05/19/2000 08:29 AM\n05/19/2000 08:29 AM\nTo: Drew Fossum/ET&S/Enron@ENRON\ncc:  \n\nSubject: Re: Emma Caplan  \n\nWell, if she\'s "very bright, personable, and enthusiastic" she certainly \nwon\'t fit in with us.   \n\nI\'ll follow up with mike and shelly on the person we were thiniking of \nsplitting for the summer - that may already be a done deal, if not it may \nmake sense to do something more permanent but we will need to look at \nbudget...\n\ni could probably use some part time help with Bret Reich exiled to Phase IV \nbut i should have a licensed lawyer for that.\n\n\n\n\n   \n\t\n\t\n\tFrom:  Drew Fossum                           05/18/2000 05:50 PM\n\t\n\nTo: Michael Moran/ET&S/Enron@ENRON, Dorothy McCoppin/FGT/Enron@ENRON, Louis \nSoldano/ET&S/Enron@ENRON, Britt

In [27]:
ds

Dataset({
    features: ['text', 'meta', 'domain'],
    num_rows: 100000
})

In [33]:
len(tokenized[0]['input_ids'])

64